<a href="https://colab.research.google.com/github/levennn19/deepstrum/blob/main/deepstrum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install demucs

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [ ]:
import librosa
import numpy as np
from google.colab import drive
from IPython.display import Audio
import soundfile as sf



#Seperation
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO


#Cleaning noise
# Install required libraries
!pip install PyWavelets
!pip install librosa

# Import libraries
import pywt

In [4]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!

drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/gdrive/MyDrive/demucs/'
out_path = '/gdrive/MyDrive/demucs_separated/'

In [6]:
#Seperation

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')
    
    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()
    
    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()
    
    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

In [8]:
# Load audio file
audio_file_seperated = "/gdrive/MyDrive/demucs_separated/htdemucs/song/other.mp3"
y, sr = librosa.load(audio_file_seperated)


# Apply wavelet transform
wavelet = 'db4'
level = 1
coeffs = pywt.wavedec(y, wavelet, level=level)

# Find the threshold
sigma = np.median(np.abs(coeffs[-level])) / 0.6745
threshold = sigma * np.sqrt(2 * np.log(len(y)))

# Apply soft thresholding to coefficients
for i in range(1, len(coeffs)):
    coeffs[i] = pywt.threshold(coeffs[i], threshold)

# Reconstruct the audio signal
audio_file_denoised = pywt.waverec(coeffs, wavelet)

# Save denoised audio file
out_path = "/gdrive/MyDrive/denoised_audio.wav"
sf.write(out_path, audio_file_denoised, sr)



In [ ]:
audio_file_original ="/gdrive/MyDrive/demucs/song.mp3"
Audio(audio_file_original,rate=sr)

In [ ]:
Audio(audio_file_seperated,rate=sr)

In [ ]:
Audio(audio_file_denoised,rate=sr)

In [12]:
audio_file_denoised ="/gdrive/MyDrive/denoised_audio.wav"

In [13]:
# Load audio file
try:
    y, sr = librosa.load(audio_file_denoised, sr=44100)
except FileNotFoundError:
    print("File not found. Please check the file path.")
    y, sr = None, None

if y is not None:
    # Extract CQT features for first 120 seconds
    duration = 120
    y_preprocessed = y[:duration*sr]
    cqt_preprocessed = np.abs(librosa.cqt(y_preprocessed, sr=sr))

In [ ]:
#tempo calculation
y,sr = librosa.load(audio_file_denoised)
tempo , beat_frames = librosa.beat.beat_track(y=y,sr=sr)
print(f"Tempo: {tempo:.2f} BPM")

In [ ]:
Audio(y_preprocessed, rate=sr)

In [16]:
# Convert CQT frequency bins to Hz
freqs = librosa.cqt_frequencies(cqt_preprocessed.shape[0], fmin=librosa.note_to_hz('C1'), bins_per_octave=12)

In [17]:
# Calculate time steps
hop_length = 512
frame_length = cqt_preprocessed.shape[1]
time_steps = librosa.frames_to_time(np.arange(frame_length), sr=sr, hop_length=hop_length)

In [18]:
# Calculate frequency at time steps of t seconds
freq_hz = []
t = 60/tempo;
window_size = int(t * sr / hop_length)  # window size in frames
for i in range(0, frame_length - window_size, window_size):
    start_idx = i
    end_idx = i + window_size
    freq_bin = np.mean(cqt_preprocessed[:, start_idx:end_idx], axis=1)
    freq_hz.append(librosa.midi_to_hz(librosa.hz_to_midi(freqs[np.argmax(freq_bin)])))


In [19]:
# Convert frequency list to array
freq_array = np.array(freq_hz)

In [ ]:
# Print frequency array
print(freq_array)

In [ ]:
number_list = [130.81, 261.63,138.59,277.18, 146.83 ,293.66, 155.56 ,311.13, 82.41 ,164.81 ,329.63, 87.31 ,174.61 ,349.23, 92.50 ,185.00 ,369.99, 98.00 ,196.00 ,392.00, 103.83 ,207.65 ,415.30, 110.00 ,220.00 ,440.00, 116.54 ,233.08 ,466.16, 123.47 ,246.94]



for input_value in freq_array:
    closest_value = None
    closest_distance = None
    for number in number_list:
        distance = abs(input_value - number)
        if closest_distance is None or distance < closest_distance:
           closest_value = number
           closest_distance = distance

    string= None
    fret=None

    for i in range(31):
        if(number_list[i]==closest_value):
            break


    if(i==0):
        string=5
        fret=3
    elif(i==1):
        string=2
        fret=1
    elif(i==2):
        string=5
        fret=4
    elif(i==3):
        string=2
        fret=2
    elif(i==4):
        string=4
        fret=0
    elif(i==5):
        string=2
        fret=3
    elif(i==6):
        string=4
        fret=1
    elif(i==7):
        string=2
        fret=4
    elif(i==8):
        string=6
        fret=0
    elif(i==9):
        string=4
        fret=2
    elif(i==10):
        string=1
        fret=0
    elif(i==11):
        string=6
        fret=1
    elif(i==12):
        string=4
        fret=3
    elif(i==13):
        string=1
        fret=1
    elif(i==14):
        string=6
        fret=2
    elif(i==15):
        string=4
        fret=4
    elif(i==16):
        string=1
        fret=2
    elif(i==17):
        string=6
        fret=3
    elif(i==18):
        string=3
        fret=0
    elif(i==19):
        string=1
        fret=3
    elif(i==20):
        string=6
        fret=4
    elif(i==21):
        string=3
        fret=1
    elif(i==22):
        string=1
        fret=4
    elif(i==23):
        string=5
        fret=0
    elif(i==24):
        string=3
        fret=2
    elif(i==25):
        string=1
        fret=5
    elif(i==26):
        string=5
        fret=1
    elif(i==27):
        string=3
        fret=3
    elif(i==28):
        string=1
        fret=6
    elif(i==29):
        string=5
        fret=2
    elif(i==30):
        string=2
        fret=0

    print(f"({string} , {fret})")